In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os

import numpy as np

from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
import kachery as ka

Autosaving every 180 seconds


In [2]:
# SET ROOT DIRECTORY WHERE DATA WILL BE SAVED
root_dir = '/media/cat/4TBSSD/data/spikingforest/hybrid_janelia/static_siprobe/rec_16c_1200s_11/'
root_dir = '/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH'


In [14]:
#LOAD RAW DATA

# Configure kachery to download data from the public database
ka.set_config(fr='default_readonly')

# Specify the recording and ground truth sorting
# The addresses files can be obtained from this link: https://spikeforest.flatironinstitute.org/recordings
# see datasets on the left hand side of this website
# Ask Cat (or Flatiron people) about which datasets to focus on 
recording_path = 'sha1dir://615aa23efde8898aa89002613e20ad59dcde42f9.hybrid_janelia/static_siprobe/rec_16c_1200s_11'

# LOAD DATA
print ("...loading recording...")
recording = AutoRecordingExtractor(recording_path, download=False)
data = recording.get_traces()
data = data.T # need to transpose from C order to Fortran order for YASS; final shape data = [n_times, n_chans]
print ("DATA:", data.shape)
#data.tofile(os.path.join(root_dir,'data.bin')  # 

print ("DONE LOADING / SAVING RAW DATA")

...loading recording...
Downloaded file (28.4 KiB) in 0.2378990650177002 sec.
Downloaded file (1.1 GiB) in 128.60074615478516 sec.
DATA: (36000000, 16)
DONE LOADING / SAVING RAW DATA


In [25]:
from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
import kachery as ka
ka.set_config(fr='default_readonly')

recording_path = '/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH'

# LOAD DATA
print ("...loading recording...")
recording = AutoRecordingExtractor(recording_path, download=False)
print ("finished loading recording")
#sorting_true_path = os.path.join(recording_path,'firings_true.mda')
#sorting = AutoSortingExtractor(sorting_true_path, samplerate=recording.get_sampling_frequency())

...loading recording...


Exception: Unable to initialize recording extractor. Unable to determine format of recording: /media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH

In [18]:
# LOAD GROUND TRUTH SPIKETIMES
print ("...loading ground truth data...")

# initialize the extractor; for some reason it needs to be initialized first
recording_path = 'sha1dir://10b2e53b6b3aa0731b763db42daa692c4e1564b0.synth_mearec_neuronexus/datasets_noise10_K20_C32/001_synth'
recording = AutoRecordingExtractor(recording_path, download=False)

# 
sorting_true_path = os.path.join(recording_path,'firings_true.mda')
sorting_true = AutoSortingExtractor(sorting_true_path)

ids = sorting_true.get_unit_ids()
print (" neuron ids: ", ids)
spike_train = np.zeros((0,2),'int32')
for id_ in ids:
    temp = sorting_true.get_unit_spike_train(id_)
    temp2 = np.zeros((temp.shape[0],2),'int32')
    temp2[:,0] = temp
    temp2[:,1] = id_
    spike_train = np.vstack((spike_train, temp2))

# convert to zero-based indexes
if np.min(spike_train[:,1]==0):
    spike_train[:,1]-=1
print (" spike train: ", spike_train)
ground_truth_dir = os.path.join(root_dir,'ground_truth')

# make ground-truth directory
try:
    os.mkdir(ground_truth_dir)
except:
    pass

#np.save(ground_truth_dir+'/spike_train_ground_truth.npy', spike_train)

print ("DONE SAVING GROUND TRUTH SPIKETIMES")

...loading ground truth data...
Downloaded 10.0 MiB in 69.53353524208069 sec from http://132.249.245.245:24342/get/sha1/c9a62ee5ae1f71097a17acb9b7aca3457161c76e?channel=public&signature=6920b21e7bbfb765fe9131d97b801d65e71807e8
Downloaded file (1.2 MiB) in 1.527402400970459 sec.
 neuron ids:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
 spike train:  [[   15944       -1]
 [   23766       -1]
 [   24017       -1]
 ...
 [17984516       18]
 [17986880       18]
 [17995807       18]]
DONE SAVING GROUND TRUTH SPIKETIMES


In [21]:
np.save('/media/cat/4TBSSD/data/spikingforest/synth_mearec_neuronexus_noise10_K20_C32/001_SYNTH/ground_truth/spike_train_gt.npy', spike_train)

In [13]:
# Visualize data
data = data.T # need to transpose from C order to Fortran order for YASS
print ("DATA (Should be [n_times, n_chans]):", data.shape)
#data.tofile(os.path.join(root_dir,'data.bin')  # might need to ravel, but not always required.

for k in range(data.shape[1]):    
    plt.plot(data[:10000,k]+k*100,c='black')
plt.show()
